In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from tensorflow.keras.optimizers import Adam
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional

import numpy as np
from numpy import array
import pandas as pd
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt

from sklearn import model_selection,naive_bayes,svm
from sklearn.linear_model import LogisticRegression

import os
import sys
import numpy as np

import gensim
from gensim.models import KeyedVectors
import time

In [2]:
df = pd.read_csv('../dataset/golbeck.csv')


In [3]:
#shuffle the dataset
from sklearn.utils import shuffle
df = shuffle(df)

In [4]:
df.dropna(inplace=True)

In [5]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}

In [6]:
import re

text = ' '.join(df['text'])
text = text.split()
freq_comm = pd.Series(text).value_counts()
rare = freq_comm[freq_comm.values == 1]

def get_clean_text(x):
    if type(x) is str:
        x = x.lower()
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        x = re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', x) 
        #regex to remove to emails
        x = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x)
        #regex to remove URLs
        x = re.sub('RT', "", x)
        #substitute the 'RT' retweet tags with empty spaces
        x = re.sub('[^A-Z a-z]+', '', x)
        #combining all the text excluding rare words.
        x = ' '.join([t for t in x.split() if t not in rare])
        return x
    else:
        return x
    
df['text'] = df['text'].apply(lambda x: get_clean_text(x)) 

In [7]:

wordembeddings = gensim.models.KeyedVectors.load_word2vec_format('../dataset/GoogleNews-vectors-negative300.bin', binary=True)

In [8]:
tokenizer = Tokenizer(num_words=2000)

In [9]:
tokenizer.fit_on_texts(df['text'])

In [10]:
sequences = tokenizer.texts_to_sequences(df['text'])

In [11]:
word_index = tokenizer.word_index


In [12]:
max_length = 0
for review_number in range(len(sequences)):
    numberofwords=len(sequences[review_number])
    if (numberofwords) > (max_length):
        max_length = numberofwords
print(max_length)

395


In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
data = pad_sequences(sequences, maxlen=max_length)
y = np.asarray(df['label'])


In [14]:
# Converting data to matrix of shape (12,300) where 12 is number of words in vocaboulry + 1 and 300 word vector for each word
unique_words = len(word_index)
total_words = unique_words + 1
skipped_words = 0
embedding_dim = 300  
embedding_matrix = np.zeros((total_words, embedding_dim))
embedding_vector=np.array(300)
for word, index in tokenizer.word_index.items():
    try:
        embedding_vector = wordembeddings[word]
    except:
        skipped_words = skipped_words+1
    pass
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
print("Embeddings Matrix shape : ",embedding_matrix.shape)

Embeddings Matrix shape :  (1212, 300)


In [15]:
len(embedding_vector)

300

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data, y, random_state = 14, test_size = 0.2, stratify = y)

In [17]:
embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

In [18]:
start=time.time()
model = Sequential()
model.add(embedding_layer)

#model.add(LSTM(128))
model.add(Bidirectional(LSTM(128)))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])
# fit network
model.fit(X_train, y_train, epochs=5, verbose=1)
end=time.time()
print(end-start)


Train on 16076 samples
Epoch 1/5
16076/16076 [==============================] - 532s 33ms/sample - loss: 0.5794 - accuracy: 0.7400
Epoch 2/5
16076/16076 [==============================] - 500s 31ms/sample - loss: 0.5491 - accuracy: 0.7518
Epoch 3/5
16076/16076 [==============================] - 499s 31ms/sample - loss: 0.5391 - accuracy: 0.7558
Epoch 4/5
16076/16076 [==============================] - 496s 31ms/sample - loss: 0.5324 - accuracy: 0.7586
Epoch 5/5
16076/16076 [==============================] - 501s 31ms/sample - loss: 0.5297 - accuracy: 0.7631
2529.7552573680878


Train on 9490 samples
Epoch 1/5
9490/9490 [==============================] - 21s 2ms/sample - loss: 0.2865 - accuracy: 0.9358
Epoch 2/5
9490/9490 [==============================] - 17s 2ms/sample - loss: 0.1855 - accuracy: 0.9456
Epoch 3/5
9490/9490 [==============================] - 19s 2ms/sample - loss: 0.1593 - accuracy: 0.9516
Epoch 4/5
9490/9490 [==============================] - 18s 2ms/sample - loss: 0.1509 - accuracy: 0.9564
Epoch 5/5
9490/9490 [==============================] - 19s 2ms/sample - loss: 0.1485 - accuracy: 0.9557s - loss:


In [20]:
y_predict=model.predict(np.array(X_test))

In [21]:
y_hat=np.where(y_predict > 0.5, 1, 0)

In [22]:
geometric_mean_score(y_test,y_hat)

C:\Users\20177465\Anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.5696287919299365

In [23]:
f1_score(y_test,y_hat)

0.4593301435406698

In [24]:
accuracy_score(y_test,y_hat)

0.9523809523809523

In [25]:
y_pred_keras = model.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)

In [26]:
auc_model = auc(fpr_keras, tpr_keras)

In [27]:
auc_model

0.8786374388030145

In [28]:
y=y_hat.flatten()

In [29]:
dt=pd.read_csv("founta_label.csv")

In [30]:
result=pd.DataFrame({'text':dt['text'],'label':y})

In [31]:
result.to_csv("results_word_bilstm.csv")